In [16]:
%run ../notebooks/01_data_and_utils.ipynb

[INFO] Using device: cuda
[INFO] Directory structure initialized

❗ GoPro dataset NOT found.
ACTION REQUIRED:
1. Download from: https://seungjunnah.github.io/Datasets/gopro
2. Extract into: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\datasets\GoPro
Expected structure:
GoPro/train/blur, GoPro/train/sharp, GoPro/test/blur, GoPro/test/sharp

❗ RealBlur dataset NOT found.
Download from: https://cg.postech.ac.kr/research/realblur/
Extract into: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\datasets\RealBlur

❗ LOL dataset NOT found.

ACTION REQUIRED:
1. Download the LOL dataset manually from one of these sources:
   - https://daooshee.github.io/BMVC2018website/
   - https://github.com/daooshee/Low-light-image-enhancement

2. Extract it into:
   C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\datasets\LOL

Expected structure:
LOL/
 ├── our485/
 │    ├── low/
 │    └── high/
 └── eval15/
      ├── low/
      └── high/


❗ ICDAR 2015 NOT found.
Register & download from: https://rrc.cv

In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Tuple

import torch


In [18]:
# Reuse utilities defined earlier
# These should already exist in kernel memory if notebook 01 was run

assert "read_image" in globals(), "Run 01_data_and_utils.ipynb first"
assert "normalize_img" in globals(), "Run 01_data_and_utils.ipynb first"


In [19]:
def laplacian_variance(gray_img: np.ndarray) -> float:
    """
    Computes variance of Laplacian (sharpness measure).
    Lower value → more blur.
    """
    lap = cv2.Laplacian(gray_img, cv2.CV_64F)
    return lap.var()


In [20]:
def compute_blur_score(
    img: np.ndarray,
    normalize: bool = True
) -> float:
    """
    Computes blur severity score.
    Higher score → more blur.
    """
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img

    var = laplacian_variance(gray)

    if not normalize:
        return var

    # Invert + normalize (heuristic)
    blur_score = 1.0 / (var + 1e-6)
    return blur_score


In [21]:
def needs_deblurring(
    img: np.ndarray,
    threshold: float
) -> Tuple[bool, float]:
    """
    Returns:
        - needs_deblur (bool)
        - blur_score (float)
    """
    score = compute_blur_score(img)
    return score > threshold, score


In [23]:
def visualize_blur_score(img_path: Path, threshold: float):
    img = read_image(img_path)
    blur_flag, score = needs_deblurring(img, threshold)

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.title(
        f"Blur score: {score:.4f}\n"
        f"{'BLURRY → deblur' if blur_flag else 'CLEAN → bypass'}"
    )
    plt.axis("off")
    plt.show()


In [24]:
def analyze_blur_distribution(
    image_dir: Path,
    max_images: int = 200
):
    scores = []

    image_paths = list(image_dir.rglob("*.jpg")) + list(image_dir.rglob("*.png"))
    image_paths = image_paths[:max_images]

    for p in image_paths:
        img = read_image(p)
        score = compute_blur_score(img)
        scores.append(score)

    scores = np.array(scores)

    plt.figure(figsize=(6,4))
    plt.hist(scores, bins=50)
    plt.xlabel("Blur score")
    plt.ylabel("Frequency")
    plt.title("Blur Score Distribution")
    plt.show()

    print(
        f"Stats → min: {scores.min():.6f}, "
        f"mean: {scores.mean():.6f}, "
        f"max: {scores.max():.6f}"
    )

    return scores


In [25]:
# Empirical starting point
BLUR_THRESHOLD = 0.001

print(f"[INFO] Using blur threshold = {BLUR_THRESHOLD}")


[INFO] Using blur threshold = 0.001


In [26]:
def blur_detection_step(frame: np.ndarray):
    """
    Pipeline-compatible blur detection step.
    """
    needs_deblur, score = needs_deblurring(frame, BLUR_THRESHOLD)
    return {
        "blur_score": score,
        "needs_deblur": needs_deblur
    }


In [27]:
def test_blur_on_video_frames(
    frames_dir: Path,
    max_frames: int = 20
):
    frame_paths = sorted(frames_dir.glob("*.jpg"))[:max_frames]

    for p in frame_paths:
        img = read_image(p)
        res = blur_detection_step(img)

        plt.figure(figsize=(3,3))
        plt.imshow(img)
        plt.title(
            f"{p.name}\n"
            f"Score: {res['blur_score']:.4f}\n"
            f"{'BLUR' if res['needs_deblur'] else 'CLEAN'}"
        )
        plt.axis("off")
        plt.show()
